In [28]:
#nbi:hide_in
import pandas_datareader as pd2
import pandas as pd
import tensorflow
import keras as ks
import matplotlib.pyplot as mp
from keras.layers import Dense,LSTM,Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping
import numpy as np
from ipywidgets import interact,interact_manual

In [57]:
#nbi:hide_in
#WE DECLARE OUR CLASSES
#THIS FUNCTION GETS, WINDOWS AND SPLITS INTO TEST/TRAIN
class Data():
    
    def __init__(self,ticker,start,end):
        self.data = pd2.DataReader(ticker,'yahoo', start, end).iloc[:,]
        
    def window(self,windows,col):
        
        #we sort the windowed array
        windows = sorted(windows)

        #we loop through the training data and restructure
        for i in range(max(windows),self.data.count()[0]):

            y_record = self.data.iloc[[i],:].loc[:,[col]]
            x_record = self.data.iloc[[i-windows[0]],:].loc[:,[col]]
            x_record.columns = str(windows[0]) + "-" + x_record.columns

            if(len(windows)>1):

                for j in range(1,len(windows)):
                    x_temp   = self.data.iloc[[i-windows[j]],:].loc[:,[col]]
                    x_temp.columns = str(windows[j]) + "-" + x_temp.columns
                    x_record = pd.concat([x_record.reset_index(drop=True), x_temp.reset_index(drop=True)], axis=1)

            if(i==max(windows)):
                x_final = x_record
                y_final = y_record
            else:
                x_final = x_final.append(x_record)
                y_final = y_final.append(y_record)

            self.features = x_final.set_index(y_final.index)
            self.response = y_final
            
    def split_data(self,period):
        self.test_features = self.features.iloc[self.features.count()[0]-period:self.features.count()[0],:]
        self.train_features = self.features.iloc[0:self.features.count()[0]-period,:]
        self.test_response = self.response.iloc[self.features.count()[0]-period:self.features.count()[0],:]
        self.train_response = self.response.iloc[0:self.features.count()[0]-period,:]
        self.data_train = self.data.iloc[0:self.data.count()[0]-period,:]
        self.data_test =  self.data.iloc[self.data.count()[0]-period:self.data.count()[0],:]

class NN():
    
    def __init__(self,neurons,activations):
        self.neurons = neurons
        self.activations = activations
     
    def create_model(self,dim,loss,optimizer):
    
        self.model = ks.Sequential()
     
        for i in range(0,len(self.neurons)):
            neuron = self.neurons[i]
            activation = self.activations[i]
        
            if(i == 0):
                self.model.add(Dense(neuron,input_dim = dim,activation = activation))
            else:
                self.model.add(Dense(neuron,activation = activation))
                
        self.model.compile(loss=loss,optimizer=optimizer)

Please enter the instrument,start date,window period and test train split of the data required!

In [71]:
#This stores variables for data extract
def data_store(core = 'NA', window = 'NA', valid = 'NA'):
    
    global ticker
    global start
    global end
    global windows
    global validation
    
    if core != 'NA':
        temp = core.split(',')
        ticker = temp[0]
        start = temp[1]
        end  = temp[2]
        
    if windows != 'NA':
        windows = list(map(int,window.split(',')))
        
    if valid != 'NA':
        validation = int(valid)


#This extracts the data
def data_build():
    
    global data

    #we create the data model
    data = Data(ticker,start,end)
    data.window(windows,'Open')
    data.split_data(validation)
    #we plot the training and test
    mp.figure(1)
    mp.plot(data.train_response)
    mp.plot(data.test_response)
    mp.title('Train and Validation Split')
    
    #we plot the windowed data
    mp.figure(2)
    mp.plot(data.train_features)
    mp.title('Training windowed data')


interact(data_store,core ='AAPL,2014-04-04,2016-04-04',window='30,60,90',valid='90')
interact_manual(data_build)

interactive(children=(Text(value='AAPL,2014-04-04,2016-04-04', description='core'), Text(value='30,60,90', des…

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

<function __main__.data_build()>

In [82]:
def neural_network_store(neurons,activations,epoch_in,batch_size_in,verbose_in):
    global  neuron_array 
    global  activation_array 
    global  epochs
    global  batch_size 
    global  verbose
    
    #organize inputs
    neuron_array = list(map(int,neurons.split(',')))
    activation_array = activations.split(',')
    epochs = int(epoch_in)
    batch_size = int(batch_size_in)
    verbose = int(verbose_in)
    
    
def neural_network_build():
    global network
    global history 
    
    network = NN(neuron_array,activation_array)
    network.create_model(dim=data.train_features.shape[1],loss='mse',optimizer='adam')
    Stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=50, verbose=1, mode='auto', baseline=None)
    history = network.model.fit(data.train_features,data.train_response.loc[:,'Open'],epochs=epochs,batch_size=batch_size,verbose=verbose,callbacks=[Stop],validation_data = (data.test_features,data.test_response.loc[:,'Open']))

    #we plot the training and test
    mp.figure(1)
    mp.plot(data.train_response.index,network.model.predict(data.train_features))
    mp.plot(data.train_response)
    mp.title('Training and Prediction')
    
    #we plot the training and test
    mp.figure(2)
    mp.plot(data.test_response.index,network.model.predict(data.test_features))
    mp.plot(data.test_response)
    mp.title('Validation and Prediction')
    
    #we plot the training and test
    mp.figure(3)
    mp.plot(history.history['val_loss'])
    mp.plot(history.history['loss'])
    mp.title('Training and Validation Error')
    
interact(neural_network_store,neurons ='3,3,1',activations = 'linear,sigmoid,linear',epoch_in = '300',batch_size_in ='2',verbose_in = '1')
interact_manual(neural_network_build)


interactive(children=(Text(value='3,3,1', description='neurons'), Text(value='linear,sigmoid,linear', descript…

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

<function __main__.neural_network_build()>

In [80]:
history.history

{'val_loss': [10499.004036458333,
  10377.608333333334,
  10037.087760416667,
  9850.107845052084,
  9712.3955078125,
  9579.778038194445,
  9449.447254774306,
  9322.698307291666,
  9197.082313368055,
  9073.835633680555],
 'loss': [13427.726438540054,
  13164.335686556695,
  12867.19288469911,
  12591.501693111455,
  12428.77001499613,
  12276.153175188661,
  12128.377754329527,
  11983.764409587848,
  11841.611325101587,
  11701.54055908959]}